Importing raindata set from IMD DATASET

# 1. Finding corret shape file

### Need to visualization of last 10 years

- tRY TO do it for karnataka until you find the correct shape file

In [23]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import HeatMap
import folium
import geopandas as gpd

def map(file):
    # Read the Shapefile using geopandas
    gdf = gpd.read_file(file)

    # Create a Folium map centered at a specific location (e.g., latitude and longitude)
    m = folium.Map(location=[0, 20], zoom_start=3)

    # Add a feature group for the mask
    mask_group = folium.FeatureGroup(name='Mask')

    # Iterate through the GeoDataFrame and add each geometry as a GeoJson object to the mask group
    for geo in gdf.geometry:
        folium.GeoJson(
            geo,
            style_function=lambda x: {'fillColor': 'red', 'color': 'red'}
        ).add_to(mask_group)

    # Add the mask group to the map
    mask_group.add_to(m)

    # Display the map
    display(m)

# Call the function with the path to your Shapefile
map('./UK/UK.shp')


Now capturing all those points which comes under that buffer file

# 2.Clip all lat long on this shpe file 

In [24]:
import xarray as xr
import geopandas as gpd
import regionmask
import numpy as np
import pandas as pd
from openpyxl import Workbook
import os
import ipdb
from scipy.interpolate import griddata
from sklearn.neighbors import KNeighborsRegressor
import time
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, display_html, HTML
import logging
from ipywidgets import Dropdown
# Ignore FutureWarnings
from tabulate import tabulate
import pymannkendall as mk
import scipy.stats as stats
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
warnings.filterwarnings("ignore", category=FutureWarning)

In [46]:
def get_clean_state_name(shapefile_path):
    state = os.path.basename(shapefile_path)  # Get the base file name
    state = state.split('.')[0]  # Remove the file extension
    if state.endswith("_buffer"):
        state = state[:-7]  # Remove "_buffer" if present
    return state
def clip_data_by_bbox(IMDDataAdress, shapefilePath, ClippedFolderPath):
    # Read the shapefile
    # print(shapefilePath)
    gdf = gpd.read_file(shapefilePath)
    # ipdb.set_trace()
    # Get the geometry
    geometry = gdf['geometry']
    # Get the bounding box
    bbox = geometry.bounds
    # Get the coordinates
    xmin = bbox['minx'][0]
    xmax = bbox['maxx'][0]
    ymin = bbox['miny'][0]
    ymax = bbox['maxy'][0]

    #Masking
    # Open the NetCDF dataset using xarray
    ds = xr.open_dataset(IMDDataAdress)
    # Define the latitude and longitude variable names to check
    possible_lat_names = ['lat', 'latitude', 'Latitude', 'LAT', 'LATITUDE', 'Lat']
    possible_lon_names = ['lon', 'longitude', 'Longitude', 'LON', 'LONGITUDE', 'Lon']

    # Initialize variables to None
    lat_var = None
    lon_var = None

    # Try to find the latitude and longitude variables
    for lat_name in possible_lat_names:
        if lat_name in ds.variables:
            lat_var = ds[lat_name]
            break

    for lon_name in possible_lon_names:
        if lon_name in ds.variables:
            lon_var = ds[lon_name]
            break

    # Apply the mask to latitude and longitude using xarray indexing
    mask_lat = (lat_var >= ymin) & (lat_var <= ymax)
    mask_lon = (lon_var >= xmin) & (lon_var <= xmax)
    # ipdb.set_trace()
    if not mask_lat.any() or not mask_lon.any():
        return None 
    data = ds.where(mask_lat & mask_lon, drop=True)
    # dataset = regionmask.Regions(name='State_Name', numbers=[0], names=gdf.State_Name[[0]], abbrevs=gdf.State[[0]], outlines=gdf.geometry[[0]])
    # Provide the correct file path

    # Finding name of state to save with
    stateName = get_clean_state_name(shapefilePath)
    Year = IMDDataAdress.split('_')[1][-4:]
    #APHRO_MA_025deg_V1101.2006.nc --> extract 2006
    stateName = "UK"
    # Construct the filename with stateName_Year.nc
    ClippedDataFile = f"{stateName}_{Year}.csv"
    # Construct full path for saving the CSV file
    csv_file_path  = os.path.join(ClippedFolderPath, ClippedDataFile)
    data = data.to_dataframe().reset_index()
    print(data)
    selected_Columns = ['TIME','LATITUDE','LONGITUDE','RAINFALL']
    data = data[selected_Columns]
    # Remove leading and trailing spaces (if any) and rename columns
    data.columns = data.columns.str.strip() 
    data = data.rename(columns={'RAINFALL':'RAINFALL', 'TIME' : 'Date'})
    data.to_csv(csv_file_path, index=False)
    # Save the .nc file for each indiividual state, for all years individually

def do_Clipping_Statewise(shapefilePath, AphoditeDataAdress, ClippedFolderPath,state):
    for dataset in os.listdir(AphoditeDataAdress):
        datasetPath = os.path.join(AphoditeDataAdress,dataset)
        clip_data_by_bbox(datasetPath, shapefilePath, ClippedFolderPath)


shapeFileAddress =['./Uk/UK.shp']

IMDRainDataAdress = "./nc"
ClippedFolderPath = "./RainAnalysis/ClippedData"

if not os.path.exists(ClippedFolderPath):
    os.makedirs(ClippedFolderPath)


for file in (shapeFileAddress):
    shapefilePath = file
    state = get_clean_state_name(shapefilePath)
    statefolder = ClippedFolderPath + '/' + state
    if not os.path.exists(statefolder):#make state folder name if not exists
        os.makedirs(statefolder)
    do_Clipping_Statewise(shapefilePath, IMDRainDataAdress, statefolder, state)

    

            TIME  LATITUDE  LONGITUDE  RAINFALL
0     2023-01-01     28.75      77.75       0.0
1     2023-01-01     28.75      78.00       0.0
2     2023-01-01     28.75      78.25       0.0
3     2023-01-01     28.75      78.50       0.0
4     2023-01-01     28.75      78.75       0.0
...          ...       ...        ...       ...
56205 2023-12-31     31.25      80.00       NaN
56206 2023-12-31     31.25      80.25       NaN
56207 2023-12-31     31.25      80.50       NaN
56208 2023-12-31     31.25      80.75       NaN
56209 2023-12-31     31.25      81.00       NaN

[56210 rows x 4 columns]


In [35]:
from netCDF4 import Dataset

# Path to the NetCDF file
file_path = './nc/RF25_ind2023_rfp25.nc'

# Open the NetCDF file
nc_file = Dataset(file_path, 'r')

# Print the dimensions (shape) of the NetCDF file
print("Dimensions (Shape):")
for dim in nc_file.dimensions:
    print(f"{dim}: {len(nc_file.dimensions[dim])}")

# Print the variables and their attributes
print("\nVariables:")
for var in nc_file.variables:
    print(f"Variable: {var}")
    print("Shape:", nc_file.variables[var].shape)
    print("Attributes:")
    for attr_name in nc_file.variables[var].ncattrs():
        print(f"\t{attr_name}: {getattr(nc_file.variables[var], attr_name)}")

# Close the NetCDF file
nc_file.close()


Dimensions (Shape):
LONGITUDE: 135
LATITUDE: 129
TIME: 365

Variables:
Variable: LONGITUDE
Shape: (135,)
Attributes:
	units: degrees_east
	point_spacing: even
	axis: X
	modulo: 360.0
	standard_name: longitude
Variable: LATITUDE
Shape: (129,)
Attributes:
	units: degrees_north
	point_spacing: even
	axis: Y
	standard_name: latitude
Variable: TIME
Shape: (365,)
Attributes:
	units: days since 1900-12-31
	axis: T
	calendar: GREGORIAN
	time_origin: 31-DEC-1900
	standard_name: time
Variable: RAINFALL
Shape: (365, 129, 135)
Attributes:
	missing_value: -999.0
	_FillValue: -999.0
	long_name: Rainfall
	units: mm
	history: From ind2023_rfp25.grd


In [47]:
import xarray as xr
import geopandas as gpd
import regionmask
import numpy as np
import pandas as pd
from openpyxl import Workbook
import os
import ipdb
def DoRainMeanMonthWise(statepath,excelsheetfolder,Months):
  # Initialize a list to store dictionaries for each point
    lat_var_names = ['lat', 'latitude', 'Latitude', 'LAT', 'LATITUDE', 'Lat']
    lon_var_names = ['lon', 'longitude', 'Longitude', 'LON', 'LONGITUDE', 'Lon']
    lat = None
    lon = None
    
    for month in Months:
        summed_data = pd.DataFrame()
        for filename in os.listdir(statepath):
            clipfile = os.path.join(statepath, filename)
            if os.path.isfile(clipfile):
                csv_file = pd.read_csv(clipfile)
                # Define the variable names for latitude and longitude
                for lat_name in lat_var_names:
                    if lat_name in csv_file.columns:
                        lat = lat_name
                        break

                for lon_name in lon_var_names:
                    if lon_name in csv_file.columns:
                        lon =  lon_name
                        break

                csv_file['Date'] = pd.to_datetime(csv_file['Date'])
                csv_file['Month'] = csv_file['Date'].dt.month.map({
                1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
                7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
                })
                #Capture all the data for a particular month
                month_year_data = csv_file[(csv_file['Month'] == month)]
                #group by lat and lon
                month_mean_data = month_year_data.groupby([lat, lon])['RAINFALL'].mean().reset_index()
                 # Calculate the mean RAINFALL for each latitude and longitude for that month
                baseFileName = os.path.basename(clipfile)
                name, extensions = baseFileName.split('.')
                name, year = name.split('_')
                point_data = {
                   'Latitude': month_mean_data[lat].values,
                   'Longitude': month_mean_data[lon].values,
                   f'RAINFALL_{year}': month_mean_data[f'RAINFALL'].values
                }
                # Convert the dictionary to a Pandas DataFrame
                point_data = pd.DataFrame(point_data, columns=['Latitude', 'Longitude', f'RAINFALL_{year}'])
                # Merge point_data into summed_data based on Latitude and Longitude
                if not summed_data.empty:
                    summed_data = pd.merge(summed_data, point_data, on=['Latitude', 'Longitude'], how='outer')
                else:
                    summed_data = point_data.copy()
        

        saveFolder = os.path.join(excelsheetfolder, name)
        if not os.path.exists(saveFolder):
            os.makedirs(saveFolder)
        excel_file = os.path.join(saveFolder, f'{name}_{month}.xlsx')
        if os.path.exists(excel_file):
            existing_data = pd.read_excel(excel_file)
            # Merge existing_data with summed_data based on Latitude and Longitude
            summed_data = pd.merge(existing_data, summed_data, on=['Latitude', 'Longitude'], how='outer')

        # Create folders and save Excel files for each month
        summed_data.to_excel(excel_file, index=False)
        
Months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
clipfolder = './RainAnalysis/ClippedData'
excelsheetfolder = './RainAnalysis/ExcelSheets'
if not os.path.exists(excelsheetfolder):
  os.makedirs(excelsheetfolder)
for state in os.listdir(clipfolder):# uk 
  statepath = os.path.join(clipfolder, state)
  name = os.path.basename(statepath)
  print(f'--------{name}- started-----')
  if os.path.isdir(statepath):#move to folder dir of clipped folder
        DoRainMeanMonthWise(statepath,excelsheetfolder,Months)
        
# it will go to each yearwise nc file and find the sum of precipitation for each point and save it in csv file

--------UK- started-----


In [48]:
directory = './RainAnalysis/ExcelSheets/UK/'

# Iterate through files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):  # Check if it is an Excel file
        filepath = os.path.join(directory, filename)
        
        # Read the Excel file
        df = pd.read_excel(filepath)
        
        # Fill null values in 'RAINFALL' column with zeros only when the cell is empty
        df.loc[df['RAINFALL_2023'].isnull(), 'RAINFALL_2023'] = 0
        
        # Save the modified DataFrame back to the Excel file
        df.to_excel(filepath, index=False)

## aNALYSIING POINTS

In [ ]:
file = 'RainAnalysis/ExcelSheets/UK/UK_Jun.xlsx'
df = pd.read_excel(file)
def map(shapefile, coordinates):
    # Read the Shapefile using geopandas
    gdf = gpd.read_file(shapefile)

    # Create a Folium map centered at a specific location (e.g., latitude and longitude)
    m = folium.Map(location=[0, 20], zoom_start=3)

    # Add a feature group for the mask
    mask_group = folium.FeatureGroup(name='Mask')

    # Iterate through the GeoDataFrame and add each geometry as a GeoJson object to the mask group
    for geo in gdf.geometry:
        folium.GeoJson(
            geo,
            style_function=lambda x: {'fillColor': 'red', 'color': 'red'}
        ).add_to(mask_group)

    # Add the mask group to the map
    mask_group.add_to(m)

    # Display the map
    display(m)

# Call the function with the path to your Shapefile
map('./UK/UK.shp', df)

In [53]:
import pandas as pd
import folium

# Read the Excel file
file = 'RainAnalysis/ExcelSheets/UK/UK_Jun.xlsx'
df = pd.read_excel(file)

# Function to create a Folium map and add points
def create_map(df, shapefile):
    # Read the Shapefile using geopandas
    gdf = gpd.read_file(shapefile)

    # Create a Folium map centered at a specific location
    m = folium.Map(location=[28.75, 77.75], zoom_start=5)  # Example location (London)

    # Add points to the map for each data entry
    for index, row in df.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']
        rainfall = row['RAINFALL_2023']

        # Add marker to the map
        folium.Marker(location=[lat, lon], popup=f'Rainfall: {rainfall}').add_to(m)

    # Add the Shapefile data to the map
    mask_group = folium.FeatureGroup(name='Mask')
    for geo in gdf.geometry:
        folium.GeoJson(
            geo,
            style_function=lambda x: {'fillColor': 'red', 'color': 'red'}
        ).add_to(mask_group)
    mask_group.add_to(m)

    # Display the map
    display(m)

# Call the function to create the map
create_map(df, './UK/UK.shp')
